# Vehicle Data Scraping

Source: https://pythoninoffice.com/get-table-data-from-web-page-using-python-pandas/
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.notna.html#pandas.DataFrame.notna  
        https://www.fueleconomy.gov/feg/PowerSearch

In [17]:
import pandas as pd
import numpy as np
import re
import bs4 as bs
import requests

fueleconomy.gov has tabulated data for all kinds of automobiles. Let's use the `read_html` function in `pandas` to scrape the website for this data. Notice these URLs are quite long. This is because I have several filters from the powersearch feature on the website activated to exclude luxury vehicle brands and focus on vehicle types that are in the UIUC fleet.

In [196]:
diesel_url = 'https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbftdiesel=Diesel&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=Diesel&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200'
electric_url = 'https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbvtelectric=Electric&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=&VehTypeSel=Electric&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200'
e85_url = 'https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbfte85=E85&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=E85&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200'
gasoline_url = 'https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbvtgasoline=Gasoline&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=&VehTypeSel=Gasoline&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=1453'

diesel = pd.read_html(diesel_url)
electric = pd.read_html(electric_url)
e85 = pd.read_html(e85_url)
gasoline = pd.read_html(gasoline_url)

The web-scraped data has all kinds of formatting issues in pandas, so we will need to clean it up a bit. I wrote the following functions to do exactly this.

In [45]:
def convert_to_string (arr):
    """
    This function converts object arrays to string arrays
    
    Parameters:
    -----------
    arr : array of objects
    
    Returns:
    --------
    str_arr : float
            Array of strings
            
    """
    str_arr = np.array([])
    for a in arr:
        str_arr = np.append(str_arr, str(a))
    return str_arr


def split_dash (s):
    """
    This function splits a string along any dashes it might have.
    
    Parameters:
    -----------
    s : string
    
    Returns:
    --------
    s_arr : strings
            Array of strings
    """
    return re.split('[–]', s)


def split_slash (s):
    """
    This function splits a string along any forward slashes it might have.
    
    Parameters:
    -----------
    s : string
    
    Returns:
    --------
    s_arr : strings
            Array of strings
    """
    return re.split('[/]', s)

def split_gal (s):
    """
    This function splits a string along 'gal' substrings
    
    Parameters:
    -----------
    s : string
    
    Returns:
    --------
    s_arr : strings
            Array of strings
    """
    return re.split('[gal]', s)

In [40]:
def get_avg_msrp (df):
    """
    This function takes the data table fron fueleconomy.gov
    and returns the average of the MSRP values listed there.
    
    Parameters:
    -----------
    df : pandas dataframe
        THe return value of pd.read_html()
    
    Returns:
    --------
    avg : float
        Average of the MSRP data
        
    """
    
    # Get rid of NaN MSRP values
    df = df[df['MSRP'].notna()]
    
    # Pick out MSRP entries that have a dollar sign
    df = df[df['MSRP'].str.contains('\$')]
    
    # Convert these to string form
    msrp_string = convert_to_string(df['MSRP'].values)
    
    # Many of the MSRPs are a given as a range between a lower and upper limit
    # of values. We want to take the average of this range.
    msrp_string = map(split_dash, msrp_string)
    msrp_num = np.array([])
    for s in msrp_string:
        if (len(s) == 2):
            s[0] = float(s[0][1:].replace(',',''))
            s[1] = float(s[1][1:].replace(',',''))
            s = (s[0] + s[1]) / 2
        else:
            s = float(s[0][1:].replace (',',''))
            
        msrp_num = np.append(msrp_num, s)
            
    return np.average(msrp_num)
    

In [120]:
def get_avg_mileage (df):
    """
    This function takes the data table fron fueleconomy.gov
    and returns the average of the mileage (gal/100mi)
    listed there.
    
    Parameters:
    -----------
    df : pandas dataframe
        The return value of pd.read_html()
    
    Returns:
    --------
    avg : float
        Average of the mileage data
        
    """
    
    # Get rid of NaN mileage values
    df = df[df['DriverMPG'].notna()]
    
    # Pick out mileage entries in the gal/100 mi formal
    df = df[df['DriverMPG'].str.contains('/100')]
    
    # Convert these to string form
    mileage_string = convert_to_string(df['DriverMPG'].values)
    
    mileage_num = np.array([])
    for s in mileage_string:
        mileage_num = np.append(mileage_num, float(re.split('[ gal]', s)[0]))
            
    return np.average(mileage_num)

With our functions defined and ready to go, we just need to call them with the relevant vehicle data:

# MSRP

In [144]:
diesel_msrp = 1e-6 * get_avg_msrp(diesel)
electric_msrp = 1e-6 * get_avg_msrp(electric)
e85_msrp = 1e-6 * get_avg_msrp(e85)
gasoline_msrp = 1e-6 * get_avg_msrp(gasoline)

print(f'Diesel: {diesel_msrp} M$')
print(f'Electric: ${electric_msrp} M$')
print(f'E85: ${e85_msrp} M$')
print(f'Gasoline: ${gasoline_msrp} M$')

Diesel: 0.037713125 M$
Electric: $0.04111 M$
E85: $0.043864098837209306 M$
Gasoline: $0.023870937499999998 M$


# Mileage

In [138]:
diesel_mileage = get_avg_mileage(diesel)
e85_mileage = get_avg_mileage(e85) #gallons per 100 miles
gasoline_mileage = get_avg_mileage(e85) #gallons per 100 miles

print(f'Diesel: {diesel_mileage}gal/100mi')
print(f'E85: {e85_mileage}gal/100mi')
print(f'Gasoline: {gasoline_mileage}gal/100mi')

Diesel: 4.1329268292682935gal/100mi
E85: 6.300387596899225gal/100mi
Gasoline: 6.300387596899225gal/100mi


# CostInvest

In [151]:
IMPGSL = 0.00222 #M$/kgal
IMPDSL = 0.00248 #M$/kgal
IMPE85 = 0.00199 #M$/kgal
lifetime_mileage = 179954 #[1]

diesel_cost = diesel_msrp / (diesel_mileage*lifetime_mileage/100)
e85_cost = e85_msrp / (e85_mileage*lifetime_mileage/100)
gasoline_cost = gasoline_msrp / (gasoline_mileage*lifetime_mileage/100)

print(f'Diesel :{diesel_cost} M$/gal')
print(f'E85: {e85_cost} M$/gal')
print(f'Gasoline: {gasoline_cost} M$/gal')

Diesel :5.0707628463053045e-06 M$/gal
E85: 3.8688370571560025e-06 M$/gal
Gasoline: 2.105429497863371e-06 M$/gal


# CostFixed

In [132]:
#assume fixed costs are 5% of MSRP annually
diesel_fixed = 1e-3 * 0.05 * diesel_msrp
electric_fixed = 1e-3 * 0.05 * electric_msrp
e85_fixed = 1e-3 * 0.05 * e85_msrp
gasoline_fixed = 1e-3 * 0.05 * gasoline_msrp

print(f':{_fixed} M$')
print(f':{_cost} M$')
print(f':{_cost} M$')

2.9699999999999998

In [142]:
(e85_mileage*lifetime_mileage/100)

11337.799496124031

In [8]:
diesel.loc[0]['Vehicle']

'2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Automatic 6-spd, Turbo, Diesel'

## References

1. US DOT. Vehicle Survivability and Travel Mileage Schedules. Technical Report. National Center for Statistics and Analysis. 2006. [link](https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/809952)

In [32]:
test = pd.read_html('https://www.fueleconomy.gov/feg/Find.do?action=sbs&id=43270')[7]

IndexError: list index out of range

In [ ]:
test

In [3]:
diesel = pd.read_html('https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbftdiesel=Diesel&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=Diesel&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200')

In [45]:
diesel[0]

,Vehicle,EPA Fuel Economy,DriverMPG,Unnamed: 3,AnnualFuelCost,MSRP,EnergyImpactScore,GreenhouseGasEmissions(tailpipe)
0,"2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom...","2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom...","2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom...","2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom...","2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom...","2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom...","2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom...","2019 Chevrolet Equinox FWD 1.6 L, 4 cyl, Autom..."
1,NaN,32 MPG 28 39 combined city/hwy city hwy ...,NaN,NaN,"$1,300","$23,800–$33,900",11.9 barrels/yr,321 grams/mile
2,NaN,32,MPG,MPG,NaN,NaN,NaN,NaN
3,NaN,32,28,39,NaN,NaN,NaN,NaN
4,NaN,combined city/hwy,city,hwy,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
487,NaN,19 MPG 18 22 combined city/hwy city hwy ...,NaN,NaN,"$2,200","$41,300–$42,900",20.1 barrels/yr,524 grams/mile
488,NaN,19,MPG,MPG,NaN,NaN,NaN,NaN
489,NaN,19,18,22,NaN,NaN,NaN,NaN
490,NaN,combined city/hwy,city,hwy,NaN,NaN,NaN,NaN


In [299]:
f = requests.get(diesel_url).text

sp = bs.BeautifulSoup(f, 'html.parser')
tb = sp.table.findall(href=testfunc)
df = pd.read_html(str(tb),encoding='utf-8', header=0)[0]
#df['href'] = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tb.find_all('a')]

TypeError: 'NoneType' object is not callable

In [387]:
slug = 'https://www.fueleconomy.gov/feg/'
def testfunc (href):
    return href and re.compile("action=sbs").search(href)
l = sp.table.find(href=testfunc) #find all the entires in the table that have links to the car page
t=l#for t in l:
ft = requests.get(slug + t['href']).text
car_page = bs.BeautifulSoup(ft, 'html.parser')
#car_page.table.

In [469]:
def testfunc2 (tag):
    return tag #and re.compile("gallons").search(tag.string)

In [441]:
car_page.find_all('td')[28]

<td class="sbsEconData">14.9 gallons</td>

In [382]:
slug + t['href']

'https://www.fueleconomy.gov/feg/Find.do?action=sbs&id=39312'

In [455]:
car_page.find_all('td')[0].name == 'td'

True

In [471]:
car_page.find_all(tag=testfunc2)

[]

In [389]:
if (car_page.find_all('td')[30] and re.compile("gallons").search(car_page.find_all('td')[30].string)):
    print('yes')

In [413]:
x = car_page.find_all('td')[28]

In [422]:
re.compile('gallons').search(x.string)

<re.Match object; span=(5, 12), match='gallons'>

In [424]:
testfunc2(x)

<re.Match object; span=(5, 12), match='gallons'>